In [1]:
import pint.models as model
import pint.toa as toa
import pint.logging
import pint.fitter
from pint.modelutils import model_ecliptic_to_equatorial
import numpy as np
import astropy.units as u
import astropy.constants as c
import pint.derived_quantities as dq
import table_utils as tu
from scipy.special import fdtrc
import matplotlib.pyplot as plt

import matplotlib as mpl
plt.rc('font',**{'family':'serif','serif':['Computer Modern Roman']})
params = {'text.usetex'       : True,
          'axes.unicode_minus': True}
mpl.rcParams.update(params)

pint.logging.setup(level="ERROR")

1

**J1816+4510 eclipsing calculations**

For J1816+4510, we see additional delays around superior conjuntion at 820 MHz, probably due to additional electron content around the companion that the signal must travel through when the pulsar eclipses. First I want to figure out the DM bump corresponding to this delay (roughly 800 microseconds).

In [2]:
def SC_dDM(dt,freq):
    """ Calculate additional column density at superior conjunction (given additional timing delay)
    
    Parameters
    ==========
    dt: quantity, additional dispersive timing delay due to plasma around companion
    freq: quantity, observing frequency of measured delay
    
    Returns
    =======
    dDM_sup: quantity, additional column density at superior conjunction (cm^-2)
    """
    Dconst = 4.148808e3*(1e6*u.Hz)**2*u.cm**3*u.s*u.pc**-1 # HBOPA, eqn. 4.6
    dDM_sup = (dt*freq**2/Dconst).to(u.pc*u.cm**-3) # extra DM corresponding to delay
    return dDM_sup.to(u.cm**-2)

To compare to a similar calculation in Freire (2005; ASPCS, p. 407), I convert DM (column density) to 1/cm^2. The result is comparable to the column density of J2051-0827 (Stappers et al. 2001).

In [3]:
dDM_1816 = SC_dDM(800.0*1e-6*u.s,820.0*(1e6*u.Hz))
print(dDM_1816)
print(dDM_1816.to(u.pc*u.cm**-3))

4.0007821153349005e+17 1 / cm2
0.12965651820956764 pc / cm3


How big might the companion be based on the duration of the eclipse? Assume inclination angle of 90 deg for now, and consider an eclipse that lasts some fraction of an orbit (based on gap between ingress-egress).

In [4]:
def psr_comp_separation(model,inc=90.0*u.deg,mp=1.4*u.solMass):
    """ Calculate separation between pulsar and binary companion
    
    Parameters
    ==========
    model: object, PINT timing model
    inc: quantity, assumed orbital inclination (optional, default = 90 deg)
    mp: quantity, assumed pulsar mass (optional, default = 1.4 Msun)
    """
    mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=inc, mp=mp)
    return (mo['A1'].quantity/np.sin(inc)).to(u.R_sun)*(1+(mp/mc))

def Rplasma(a, frac_eclipse):
    """ Estimate companion (plasma?) radius based on extent of eclipse
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    frac_eclipse: float, fraction of the orbit where pulsar signal is obscured
    
    Returns
    =======
    Rplasma: quantity, companion's plasma(?) radius (Solar Radii)
    """
    print(f"Nominal pulsar-companion separation: {a.value:.4f} Rsun")
    ing2sup = frac_eclipse*180.0*u.deg # angle from ingress to superior conjunction
    Rplasma = a*np.sin(ing2sup)
    print(f"Companion's estimated plasma radius: {Rplasma.value:.4f} Rsun ({Rplasma.to(u.km).value:.0f} km)")
    return Rplasma

frac_eclipse = 0.04

par_path = f"data/J1816+4510_eclipseDMX.par"
tim_path = f"data/J1816+4510_eclipseDMX.tim"
mo = model.get_model(par_path)
to = toa.get_TOAs(tim_path,model=mo)
fo = pint.fitter.WLSFitter(to,mo)
x = fo.fit_toas()
a = psr_comp_separation(mo, mp=1.64*u.solMass) # using lower limit on M_psr from Clark et al. (2023)
print(f"J1816+4510 is obscured for {frac_eclipse:.0%} of an orbit...")
rp = Rplasma(a,frac_eclipse)

J1816+4510 is obscured for 4% of an orbit...
Nominal pulsar-companion separation: 2.6041 Rsun
Companion's estimated plasma radius: 0.3264 Rsun (227061 km)


I can use the estimated companion's plasma radius to get some idea of the electron density near the companion (using the additional column density around superior conjunction). This is probably ~order of magnitude.

In [5]:
(dDM_1816/rp).to(u.cm**-3)

<Quantity 17619863.47492632 1 / cm3>

Next, I want to use the equation from Eggleton et al. (1983) to estimate the size of the companion's Roche Lobe, and thus, the path length the pulsar might be encountering around superior conjunction. Do this for a few possible inclination angles and assume pulsar mass is 1.64 solar masses.

In [6]:
def RL_Eggleton(a,mc,mp=1.4*u.solMass):
    """ Calculate Roche Lobe size in solar radii (see Eggleton 1983)
    
    Parameters
    ==========
    a: quantity, separation between the pulsar and companion
    mc: quantity, companion mass (solar masses)
    mp: pulsar mass (default: 1.4 Msun), optional
    
    Returns
    =======
    R_L: quantity, Roche Lobe size (solar radii)
    """
    q = mc/mp
    R_L = 0.49*a*q**(2.0/3)/(0.6*q**(2.0/3)+np.log(1.0+q**(1.0/3)))
    return R_L.to(u.R_sun)

incs = [90.0,85.0,80.0,75.0] # trial inclination angles, degrees

mo = model.get_model(par_path)
to = toa.get_TOAs(tim_path,model=mo)
fo = pint.fitter.WLSFitter(to,mo)
x = fo.fit_toas()

for inc in incs:
    mp = 1.64*u.M_sun
    a = psr_comp_separation(mo,inc=inc*u.deg,mp=mp)
    mc = dq.companion_mass(mo['PB'].quantity, mo['A1'].quantity, i=inc*u.deg, mp=mp)
    R_L = RL_Eggleton(a,mc,mp=mp)
    RlRp_ratio = R_L/rp

    mc_str = f"{mc.value:.4f}"
    q_str = f"{mc/mp:.4f}"
    a_str = f"{a.value:.4f}"
    rl_str = f"{R_L.value:.5f}"
    rlrp_str = f"{RlRp_ratio:.2f}"

    print(f"  i = {inc} deg; mc = {mc_str} Msun; q = {q_str}; a = {a_str} Rsun; RL = {rl_str} Rsun; RL/Rp = {rlrp_str}")

  i = 90.0 deg; mc = 0.1792 Msun; q = 0.1093; a = 2.6041 Rsun; RL = 0.55252 Rsun; RL/Rp = 1.69
  i = 85.0 deg; mc = 0.1800 Msun; q = 0.1097; a = 2.6044 Rsun; RL = 0.55324 Rsun; RL/Rp = 1.70
  i = 80.0 deg; mc = 0.1822 Msun; q = 0.1111; a = 2.6055 Rsun; RL = 0.55544 Rsun; RL/Rp = 1.70
  i = 75.0 deg; mc = 0.1860 Msun; q = 0.1134; a = 2.6073 Rsun; RL = 0.55917 Rsun; RL/Rp = 1.71


Therefore (unlike the systems considered in Freire 2005), the Roche Lobe (R_L above) is somewhat larger than the size of the plasma cloud (R_p), so matter responsible for increased dispersive delays is bound to the companion.